# Threshold day analysis

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
%matplotlib inline

cities = {'Bismarck, ND': (-100.773703, 46.801942),
          'Minneapolis, MN': (-93.2650, 44.9778),
          'Albany, NY': (-73.7562, 42.6526),
          'Omaha, NE': (-95.9345, 41.2565),
          'Columbus, OH': (-82.9988, 39.9612),
          'Denver, CO':(-104.9903, 39.7392),
          'St. Louis, MO': (-90.1994, 38.6270),
          'Charlotte, NC': (-80.8431, 35.2271),
          'Oklahoma City, OK':(-97.5164, 35.4676),
          'Tuscaloosa, AL': (-87.5692, 33.2098), 
          'San Antonio, TX': (-98.4936, 29.4241), 
          'Orlando, FL': (-81.3792, 28.5383), 
          }

out_dir = "../figures/threshold/"

# Read in the lat/lon data from the NAM 212 grid.

# Run through the city lats and lons, find the closest grid to that point, and save it in a dictionary

In [2]:
coords = xr.open_dataset('../data/nam212.nc')

lats = coords.gridlat_212.values
lons = coords.gridlon_212.values

from scipy.spatial import KDTree
import numpy as np

city_lookup = {}

pts = np.stack([lons.ravel(), lats.ravel()], axis=1)

flons, flats = pts[:, 0], pts[:, 1]

test = np.zeros(shape=(lats.shape))

tree = KDTree(list(zip(flons, flats)))

for key, value in cities.items():
    
    loc = tree.query(value)
    
    idx = np.unravel_index(loc[1], dims=lats.shape)
    
    test[idx] = 1
    
    city_lookup[key] = idx
    
    print(idx, lons[idx], lats[idx], value)
    
city_lookup

(82, 92) -100.91847 46.8055 (-100.773703, 46.801942)
(76, 108) -93.06891 44.845203 (-93.265, 44.9778)
(73, 149) -73.676025 42.597034 (-73.7562, 42.6526)
(66, 102) -95.933105 41.39053 (-95.9345, 41.2565)
(63, 130) -83.031204 39.93604 (-82.9988, 39.9612)
(62, 82) -105.10153 39.699127 (-104.9903, 39.7392)
(58, 115) -90.01058 38.50195 (-90.1994, 38.627)
(50, 136) -80.90761 35.120552 (-80.8431, 35.2271)
(49, 98) -97.64347 35.33841 (-97.5164, 35.4676)
(44, 121) -87.63795 33.391075 (-87.5692, 33.2098)
(33, 96) -98.35003 29.529873 (-98.4936, 29.4241)
(32, 137) -81.27024 28.59184 (-81.3792, 28.5383)


C:\Users\ahaberlie1\AppData\Local\Continuum\anaconda3\envs\r37\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: 'shape' argument should be used instead of 'dims'


{'Bismarck, ND': (82, 92),
 'Minneapolis, MN': (76, 108),
 'Albany, NY': (73, 149),
 'Omaha, NE': (66, 102),
 'Columbus, OH': (63, 130),
 'Denver, CO': (62, 82),
 'St. Louis, MO': (58, 115),
 'Charlotte, NC': (50, 136),
 'Oklahoma City, OK': (49, 98),
 'Tuscaloosa, AL': (44, 121),
 'San Antonio, TX': (33, 96),
 'Orlando, FL': (32, 137)}

# Calculate mean number of days per year in which the city was on or within the 5%, 30%, 45%, and 10% significant tornado PPer contour

In [3]:
import pandas as pd

dset = xr.open_dataset("../data/tor_day_footprints_79-18_sid-212_grid.nc")

table = {'city':[], 'pp_05':[], 'pp_30':[], 'pp_45':[], 's_pp_10':[]}

for name, value in cities.items():
    
    print(name)
    
    table['city'].append(name)
    
    for var in ['pp_05', 'pp_30', 'pp_45']:

        y, x = city_lookup[name]

        vals = dset[var].sel(y=y, x=x)
        vals = vals.groupby('time.year').sum('time')
        vals = vals.mean('year')
        
        table[var].append(vals.values)

dset = xr.open_dataset("../data/sig_tor_day_footprints_79-18_sid-212_grid.nc")

for name, value in cities.items():
    
    for var in ['pp_10']:

        y, x = city_lookup[name]

        vals = dset[var].sel(y=y, x=x)
        vals = vals.groupby('time.year').sum('time')
        vals = vals.mean('year')
        
        table["s_" + var].append(vals.values)
        
df = pd.DataFrame.from_dict(table)

df.to_csv(out_dir + "mean_tor_day_risk.csv")

df

Bismarck, ND
Minneapolis, MN
Albany, NY
Omaha, NE
Columbus, OH
Denver, CO
St. Louis, MO
Charlotte, NC
Oklahoma City, OK
Tuscaloosa, AL
San Antonio, TX
Orlando, FL


,city,pp_05,pp_30,pp_45,s_pp_10
0,"Bismarck, ND",4.05,0.125,0.025,0.025
1,"Minneapolis, MN",5.6,0.775,0.4,0.4
2,"Albany, NY",1.925,0.2,0.075,0.125
3,"Omaha, NE",7.375,1.25,0.625,0.65
4,"Columbus, OH",4.625,0.475,0.2,0.4
5,"Denver, CO",8.325,0.7,0.2,0.15
6,"St. Louis, MO",5.95,1.175,0.575,0.775
7,"Charlotte, NC",4.5,0.525,0.325,0.425
8,"Oklahoma City, OK",7.625,1.475,0.925,0.7
9,"Tuscaloosa, AL",6.675,1.225,0.6,0.925


# Calculate mean number of days per year in which the city was on or within the 15%, 30%, 60%, and 10% significant hail PPer contour

In [4]:
import pandas as pd

dset = xr.open_dataset("../data/hail_day_footprints_79-18_sid-212_grid.nc")

table = {'city':[], 'pp_15':[], 'pp_30':[], 'pp_60':[], 's_pp_10':[]}

for name, value in cities.items():
    
    print(name)
    
    table['city'].append(name)
    
    for var in ['pp_15', 'pp_30', 'pp_60']:

        y, x = city_lookup[name]

        vals = dset[var].sel(y=y, x=x)
        vals = vals.groupby('time.year').sum('time')
        vals = vals.mean('year')
        
        table[var].append(vals.values)

dset = xr.open_dataset("../data/sig_hail_day_footprints_79-18_sid-212_grid.nc")

for name, value in cities.items():
    
    for var in ['pp_10']:

        y, x = city_lookup[name]

        vals = dset[var].sel(y=y, x=x)
        vals = vals.groupby('time.year').sum('time')
        vals = vals.mean('year')
        
        table["s_" + var].append(vals.values)
        
df = pd.DataFrame.from_dict(table)

df.to_csv(out_dir + "mean_hail_day_risk.csv")

df

Bismarck, ND
Minneapolis, MN
Albany, NY
Omaha, NE
Columbus, OH
Denver, CO
St. Louis, MO
Charlotte, NC
Oklahoma City, OK
Tuscaloosa, AL
San Antonio, TX
Orlando, FL


,city,pp_15,pp_30,pp_60,s_pp_10
0,"Bismarck, ND",9.225,4.45,1.525,1.275
1,"Minneapolis, MN",11.275,6.775,3.525,1.575
2,"Albany, NY",6.7,3.95,1.875,0.4
3,"Omaha, NE",18.575,11.075,5.825,2.275
4,"Columbus, OH",10.575,5.8,2.525,0.525
5,"Denver, CO",14.7,7.875,3.35,1.525
6,"St. Louis, MO",11.3,6.65,3.425,1.225
7,"Charlotte, NC",13.975,7.95,3.325,0.625
8,"Oklahoma City, OK",25.325,17.0,9.5,3.8
9,"Tuscaloosa, AL",9.05,5.55,2.625,0.875


# Calculate mean number of days per year in which the city was on or within the 15%, 30%, 60%, and 10% significant wind PPer contour

In [5]:
import pandas as pd

dset = xr.open_dataset("../data/wind_day_footprints_79-18_sid-212_grid.nc")

table = {'city':[], 'pp_15':[], 'pp_30':[], 'pp_60':[], 's_pp_10':[]}

for name, value in cities.items():
    
    print(name)
    
    table['city'].append(name)
    
    for var in ['pp_15', 'pp_30', 'pp_60']:

        y, x = city_lookup[name]

        vals = dset[var].sel(y=y, x=x)
        vals = vals.groupby('time.year').sum('time')
        vals = vals.mean('year')
        
        table[var].append(vals.values)

dset = xr.open_dataset("../data/sig_wind_day_footprints_79-18_sid-212_grid.nc")

for name, value in cities.items():
    
    for var in ['pp_10']:

        y, x = city_lookup[name]

        vals = dset[var].sel(y=y, x=x)
        vals = vals.groupby('time.year').sum('time')
        vals = vals.mean('year')
        
        table["s_" + var].append(vals.values)
        
df = pd.DataFrame.from_dict(table)

df.to_csv(out_dir + "mean_wind_day_risk.csv")

df

Bismarck, ND
Minneapolis, MN
Albany, NY
Omaha, NE
Columbus, OH
Denver, CO
St. Louis, MO
Charlotte, NC
Oklahoma City, OK
Tuscaloosa, AL
San Antonio, TX
Orlando, FL


,city,pp_15,pp_30,pp_60,s_pp_10
0,"Bismarck, ND",4.95,2.375,0.925,1.575
1,"Minneapolis, MN",9.45,5.9,3.3,1.525
2,"Albany, NY",11.0,7.525,4.45,0.825
3,"Omaha, NE",12.7,7.525,3.7,2.825
4,"Columbus, OH",14.65,9.5,5.3,0.9
5,"Denver, CO",2.275,0.55,0.15,0.3
6,"St. Louis, MO",12.875,7.925,4.05,1.425
7,"Charlotte, NC",20.6,14.0,7.25,1.55
8,"Oklahoma City, OK",15.225,8.175,3.6,3.125
9,"Tuscaloosa, AL",13.675,7.95,3.475,1.225


# 1 Day Tornado

In [6]:
import pandas as pd

dset = xr.open_dataset("../data/tor_day_footprints_79-18_sid-212_grid.nc")

tor_area = dset.sum(('x', 'y'))

dres = {'Rank':list(range(1, 11)), 
        '2%':[], '5%':[], '10%':[], 
        '15%':[], '30%':[], '45%':[], 
        '60%':[]}

for thresh in ['pp_02', 'pp_05', 'pp_10', 'pp_15', 
               'pp_30', 'pp_45', 'pp_60']:

    dset_ = tor_area.sortby(tor_area[thresh], ascending=False)
    
    dtimes = [str(x.date()) for x in pd.to_datetime(dset_[thresh].time.values[:10])]
    areas = ["{:,}".format(int(1600*x)) for x in dset_[thresh].values[:10]]

    for date, area in list(zip(dtimes, areas)):
        
        if area == '0':
            dres[str(int(thresh[3:])) + "%"].append("-")
        else:
            dres[str(int(thresh[3:])) + "%"].append(date + " (" + area + ")")
        
df_table = pd.DataFrame.from_dict(dres)
df_table = df_table.set_index('Rank')
df_table[list(dres.keys())[1:]]

df_table.to_csv(out_dir + "tor_top10_areas.csv")

df_table

,2%,5%,10%,15%,30%,45%,60%
Rank,,,,,,,
1,"2011-04-27 (1,942,400)","2011-04-27 (1,640,000)","2011-04-27 (1,286,400)","2011-04-27 (1,118,400)","2011-04-27 (835,200)","2011-04-27 (633,600)","2011-04-27 (473,600)"
2,"2011-05-25 (1,812,800)","2011-05-25 (1,452,800)","2011-05-25 (1,140,800)","2011-05-25 (913,600)","2011-04-26 (622,400)","2011-04-26 (502,400)","2011-04-26 (403,200)"
3,"1994-04-26 (1,787,200)","2004-05-30 (1,308,800)","2004-05-30 (1,025,600)","2004-05-30 (819,200)","2011-05-25 (574,400)","2003-05-04 (438,400)","2003-05-04 (344,000)"
4,"2011-04-26 (1,771,200)","1994-04-26 (1,305,600)","2003-05-06 (940,800)","2011-04-26 (808,000)","2004-05-30 (568,000)","2007-05-05 (419,200)","1999-01-21 (305,600)"
5,"1981-06-21 (1,737,600)","2003-05-06 (1,256,000)","2002-11-10 (936,000)","2002-11-10 (760,000)","2003-05-04 (542,400)","2004-05-30 (398,400)","2007-05-05 (300,800)"
6,"2003-05-06 (1,648,000)","1982-04-02 (1,251,200)","1994-04-26 (931,200)","2004-05-29 (723,200)","2007-05-05 (536,000)","2011-05-25 (369,600)","2010-06-17 (297,600)"
7,"1993-06-08 (1,643,200)","2011-04-26 (1,241,600)","2011-04-26 (923,200)","2010-10-26 (721,600)","2008-02-05 (478,400)","2008-02-05 (366,400)","2008-02-05 (296,000)"
8,"1982-04-02 (1,624,000)","1993-06-08 (1,208,000)","1982-04-02 (905,600)","2003-05-04 (720,000)","2004-05-29 (475,200)","1999-01-21 (361,600)","2012-03-02 (268,800)"
9,"2004-05-30 (1,587,200)","2002-11-10 (1,206,400)","1995-05-18 (872,000)","2003-05-06 (712,000)","2002-11-10 (467,200)","2004-11-23 (353,600)","2011-04-19 (264,000)"


### dset.sum(('x', 'y')) is equivalent to the following code, but for every day

In [7]:
a = dset.sel(time='2011-04-27')

for v in ['pp_05', 'pp_30', 'pp_45']:
    print(v, np.sum(a[v].values) * 1600)

pp_05 1640000
pp_30 835200
pp_45 633600


# Significant tornado

In [8]:
import pandas as pd

dset = xr.open_dataset("../data/sig_tor_day_footprints_79-18_sid-212_grid.nc")

tor_area = dset.sum(('x', 'y'))

dres = {'Rank':list(range(1, 11)), 
        '2%':[], '5%':[], '10%':[], 
        '15%':[], '30%':[], '45%':[], 
        '60%':[]}

for thresh in ['pp_02', 'pp_05', 'pp_10', 'pp_15', 
               'pp_30', 'pp_45', 'pp_60']:

    dset_ = tor_area.sortby(tor_area[thresh], ascending=False)
    
    dtimes = [str(x.date()) for x in pd.to_datetime(dset_[thresh].time.values[:10])]
    areas = ["{:,}".format(int(1600*x)) for x in dset_[thresh].values[:10]]

    for date, area in list(zip(dtimes, areas)):
        
        if area == '0':
            dres[str(int(thresh[3:])) + "%"].append("-")
        else:
            dres[str(int(thresh[3:])) + "%"].append(date + " (" + area + ")")
        
df_table = pd.DataFrame.from_dict(dres)
df_table = df_table.set_index('Rank')
df_table[list(dres.keys())[1:]]

df_table.to_csv(out_dir + "sig_tor_top10_areas.csv")

df_table

,2%,5%,10%,15%,30%,45%,60%
Rank,,,,,,,
1,"2011-04-27 (1,265,600)","2011-04-27 (971,200)","2011-04-27 (672,000)","2011-04-27 (502,400)","2011-04-27 (345,600)","2011-04-27 (257,600)","2011-04-27 (200,000)"
2,"1990-03-13 (1,041,600)","1990-03-13 (784,000)","1991-04-26 (550,400)","1990-03-13 (414,400)","2013-11-17 (241,600)","2013-11-17 (164,800)","1990-06-02 (126,400)"
3,"1980-04-07 (1,025,600)","1991-04-26 (745,600)","1990-03-13 (544,000)","2003-05-04 (408,000)","1990-06-02 (208,000)","1990-06-02 (158,400)","2013-11-17 (108,800)"
4,"1982-04-02 (995,200)","1980-04-07 (697,600)","2003-05-04 (502,400)","1984-06-07 (400,000)","1984-06-07 (201,600)","1992-06-16 (137,600)","1992-06-16 (97,600)"
5,"1991-04-26 (961,600)","1992-11-22 (680,000)","1984-06-07 (488,000)","1991-04-26 (396,800)","2008-02-05 (198,400)","1985-05-31 (123,200)","1985-05-31 (96,000)"
6,"1992-11-22 (894,400)","1982-04-02 (678,400)","2012-03-02 (478,400)","2012-03-02 (384,000)","1992-06-16 (198,400)","1997-03-01 (120,000)","2011-04-15 (83,200)"
7,"2002-11-10 (880,000)","2002-11-10 (652,800)","1992-11-22 (465,600)","2013-11-17 (353,600)","2012-03-02 (196,800)","2011-04-15 (113,600)","1997-03-01 (75,200)"
8,"1979-04-11 (872,000)","2012-03-02 (651,200)","2002-11-10 (464,000)","2002-11-10 (350,400)","2003-05-04 (185,600)","2008-02-05 (108,800)","2011-04-26 (57,600)"
9,"1996-04-19 (857,600)","2003-05-04 (638,400)","1982-04-02 (460,800)","1982-04-02 (348,800)","1992-11-22 (184,000)","1984-03-28 (100,800)","1999-01-21 (57,600)"


# Do the same for hail

In [9]:
import pandas as pd

dset = xr.open_dataset("../data/hail_day_footprints_79-18_sid-212_grid.nc")

tor_area = dset.sum(('x', 'y'))

dres = {'Rank':list(range(1, 11)), 
        '2%':[], '5%':[], '10%':[], 
        '15%':[], '30%':[], '45%':[], 
        '60%':[]}

for thresh in ['pp_02', 'pp_05', 'pp_10', 'pp_15', 
               'pp_30', 'pp_45', 'pp_60']:

    dset_ = tor_area.sortby(tor_area[thresh], ascending=False)
    
    dtimes = [str(x.date()) for x in pd.to_datetime(dset_[thresh].time.values[:10])]
    areas = ["{:,}".format(int(1600*x)) for x in dset_[thresh].values[:10]]

    for date, area in list(zip(dtimes, areas)):
        
        if area == '0':
            dres[str(int(thresh[3:])) + "%"].append("-")
        else:
            dres[str(int(thresh[3:])) + "%"].append(date + " (" + area + ")")
        
df_table = pd.DataFrame.from_dict(dres)
df_table = df_table.set_index('Rank')
df_table[list(dres.keys())[1:]]

df_table.to_csv(out_dir + "hail_top10_areas.csv")

df_table

,2%,5%,10%,15%,30%,45%,60%
Rank,,,,,,,
1,"2008-06-20 (4,777,600)","2008-06-22 (3,830,400)","2008-06-22 (2,979,200)","2008-06-22 (2,588,800)","2008-06-22 (2,004,800)","2008-06-22 (1,648,000)","2008-06-22 (1,323,200)"
2,"2008-06-22 (4,739,200)","2008-06-20 (3,716,800)","2005-06-06 (2,934,400)","2005-06-06 (2,475,200)","2009-06-15 (1,681,600)","2009-06-15 (1,395,200)","2003-05-09 (1,224,000)"
3,"2005-06-06 (4,635,200)","2005-06-06 (3,715,200)","2008-07-22 (2,798,400)","2008-07-22 (2,352,000)","2008-05-25 (1,588,800)","2003-05-09 (1,358,400)","2004-05-21 (1,187,200)"
4,"2008-07-22 (4,622,400)","2008-07-22 (3,627,200)","2008-06-20 (2,788,800)","2008-05-31 (2,264,000)","2005-06-06 (1,585,600)","2004-05-21 (1,353,600)","2009-06-15 (1,179,200)"
5,"2008-06-10 (4,446,400)","2008-06-10 (3,489,600)","2008-05-31 (2,688,000)","2008-06-19 (2,224,000)","2002-06-04 (1,584,000)","2008-05-25 (1,345,600)","2006-04-02 (1,145,600)"
6,"2008-06-21 (4,276,800)","2008-06-21 (3,329,600)","2008-06-10 (2,638,400)","2008-06-20 (2,211,200)","2004-05-24 (1,582,400)","2011-05-22 (1,281,600)","2006-04-07 (1,139,200)"
7,"2008-05-31 (4,035,200)","2008-05-31 (3,323,200)","2008-06-19 (2,619,200)","2009-06-17 (2,192,000)","2004-05-21 (1,580,800)","2004-05-24 (1,272,000)","2008-05-25 (1,137,600)"
8,"2008-06-23 (4,022,400)","2008-06-19 (3,212,800)","2009-06-17 (2,507,200)","2009-06-15 (2,190,400)","2011-05-22 (1,560,000)","2006-04-02 (1,244,800)","2011-05-22 (1,091,200)"
9,"2008-06-19 (3,950,400)","2008-06-23 (3,187,200)","2008-06-21 (2,491,200)","2002-06-04 (2,155,200)","2003-05-09 (1,555,200)","2006-04-07 (1,220,800)","2005-05-08 (1,068,800)"


# Significant hail

In [10]:
import pandas as pd

dset = xr.open_dataset("../data/sig_hail_day_footprints_79-18_sid-212_grid.nc")

tor_area = dset.sum(('x', 'y'))

dres = {'Rank':list(range(1, 11)), 
        '2%':[], '5%':[], '10%':[], 
        '15%':[], '30%':[], '45%':[], 
        '60%':[]}

for thresh in ['pp_02', 'pp_05', 'pp_10', 'pp_15', 
               'pp_30', 'pp_45', 'pp_60']:

    dset_ = tor_area.sortby(tor_area[thresh], ascending=False)
    
    dtimes = [str(x.date()) for x in pd.to_datetime(dset_[thresh].time.values[:10])]
    areas = ["{:,}".format(int(1600*x)) for x in dset_[thresh].values[:10]]

    for date, area in list(zip(dtimes, areas)):
        
        if area == '0':
            dres[str(int(thresh[3:])) + "%"].append("-")
        else:
            dres[str(int(thresh[3:])) + "%"].append(date + " (" + area + ")")
        
df_table = pd.DataFrame.from_dict(dres)
df_table = df_table.set_index('Rank')
df_table[list(dres.keys())[1:]]

df_table.to_csv(out_dir + "sig_hail_top10_areas.csv")

df_table

,2%,5%,10%,15%,30%,45%,60%
Rank,,,,,,,
1,"2011-05-22 (1,614,400)","2011-05-22 (1,137,600)","2011-05-22 (782,400)","2011-05-22 (571,200)","2014-06-03 (339,200)","2014-06-03 (240,000)","2012-03-02 (160,000)"
2,"2017-05-27 (1,304,000)","2017-05-27 (958,400)","2017-05-27 (684,800)","2017-05-27 (569,600)","2012-03-02 (337,600)","2012-03-02 (220,800)","2014-06-03 (156,800)"
3,"2009-06-17 (1,302,400)","2011-05-25 (956,800)","2011-05-25 (627,200)","2014-06-03 (492,800)","2017-05-27 (305,600)","2006-04-13 (193,600)","2006-04-13 (152,000)"
4,"2009-06-15 (1,276,800)","2009-06-17 (892,800)","2008-05-25 (585,600)","2012-03-02 (489,600)","1996-04-21 (296,000)","1996-04-21 (187,200)","2017-05-16 (134,400)"
5,"2008-06-02 (1,252,800)","2008-06-02 (880,000)","2012-03-02 (571,200)","2011-04-19 (449,600)","2011-04-19 (280,000)","2011-04-09 (180,800)","2016-04-11 (124,800)"
6,"2011-05-25 (1,246,400)","2011-05-10 (828,800)","2009-06-17 (571,200)","2008-05-25 (448,000)","2011-04-09 (278,400)","2017-05-16 (177,600)","2011-04-09 (123,200)"
7,"2015-04-08 (1,160,000)","2015-04-08 (812,800)","2014-06-03 (569,600)","2011-05-25 (446,400)","2011-05-22 (276,800)","2011-05-24 (176,000)","2011-05-24 (121,600)"
8,"2003-05-09 (1,140,800)","2003-05-09 (798,400)","2008-06-02 (566,400)","2011-04-09 (444,800)","2011-05-24 (267,200)","1993-06-02 (158,400)","2009-04-10 (116,800)"
9,"2012-07-01 (1,118,400)","2008-05-25 (787,200)","2011-05-10 (564,800)","1996-04-21 (441,600)","2006-04-13 (246,400)","2016-04-11 (150,400)","2006-03-12 (113,600)"


# Do the same for wind

In [11]:
import pandas as pd

dset = xr.open_dataset("../data/wind_day_footprints_79-18_sid-212_grid.nc")

tor_area = dset.sum(('x', 'y'))

dres = {'Rank':list(range(1, 11)), 
        '2%':[], '5%':[], '10%':[], 
        '15%':[], '30%':[], '45%':[], 
        '60%':[]}

for thresh in ['pp_02', 'pp_05', 'pp_10', 'pp_15', 
               'pp_30', 'pp_45', 'pp_60']:

    dset_ = tor_area.sortby(tor_area[thresh], ascending=False)
    
    dtimes = [str(x.date()) for x in pd.to_datetime(dset_[thresh].time.values[:10])]
    areas = ["{:,}".format(int(1600*x)) for x in dset_[thresh].values[:10]]

    for date, area in list(zip(dtimes, areas)):
        
        if area == '0':
            dres[str(int(thresh[3:])) + "%"].append("-")
        else:
            dres[str(int(thresh[3:])) + "%"].append(date + " (" + area + ")")
        
df_table = pd.DataFrame.from_dict(dres)
df_table = df_table.set_index('Rank')
df_table[list(dres.keys())[1:]]

df_table.to_csv(out_dir + "wind_top10_areas.csv")

df_table

,2%,5%,10%,15%,30%,45%,60%
Rank,,,,,,,
1,"2005-06-06 (4,548,800)","2005-06-06 (3,784,000)","2005-06-06 (3,044,800)","2016-07-06 (2,611,200)","2011-04-04 (2,051,200)","2011-04-04 (1,864,000)","2011-04-04 (1,744,000)"
2,"2008-07-22 (4,496,000)","2008-07-22 (3,712,000)","2008-07-22 (2,995,200)","2008-07-22 (2,560,000)","2016-07-06 (2,017,600)","2016-07-06 (1,676,800)","2016-07-06 (1,459,200)"
3,"2008-06-21 (4,342,400)","2012-07-04 (3,414,400)","2016-07-06 (2,896,000)","2005-06-06 (2,556,800)","2011-06-21 (1,814,400)","2011-06-21 (1,550,400)","2011-06-21 (1,350,400)"
4,"2015-06-30 (4,292,800)","2016-07-06 (3,340,800)","2016-07-14 (2,715,200)","2016-07-14 (2,350,400)","2008-07-22 (1,763,200)","2018-06-28 (1,492,800)","2018-06-28 (1,324,800)"
5,"2011-07-22 (4,169,600)","2006-07-28 (3,339,200)","2012-07-04 (2,680,000)","2011-04-04 (2,334,400)","2012-07-01 (1,761,600)","2012-07-01 (1,484,800)","2012-07-01 (1,312,000)"
6,"2012-07-04 (4,156,800)","2015-06-30 (3,300,800)","2006-07-28 (2,648,000)","2012-07-01 (2,297,600)","2018-06-28 (1,726,400)","2006-06-22 (1,393,600)","2004-05-30 (1,265,600)"
7,"2006-07-28 (4,152,000)","2008-06-26 (3,278,400)","2015-06-30 (2,612,800)","2006-07-28 (2,259,200)","2006-06-22 (1,712,000)","2004-05-30 (1,369,600)","2016-06-17 (1,193,600)"
8,"2008-06-26 (4,076,800)","2016-07-14 (3,248,000)","2012-07-01 (2,593,600)","2015-06-30 (2,233,600)","2016-07-14 (1,643,200)","2016-06-17 (1,366,400)","2012-06-29 (1,088,000)"
9,"2014-07-14 (4,054,400)","2011-07-22 (3,236,800)","2008-06-26 (2,592,000)","2012-07-04 (2,214,400)","2016-06-17 (1,633,600)","2015-06-30 (1,304,000)","2006-06-22 (1,068,800)"


# Significant wind

In [12]:
import pandas as pd

dset = xr.open_dataset("../data/sig_wind_day_footprints_79-18_sid-212_grid.nc")

tor_area = dset.sum(('x', 'y'))

dres = {'Rank':list(range(1, 11)), 
        '2%':[], '5%':[], '10%':[], 
        '15%':[], '30%':[], '45%':[], 
        '60%':[]}

for thresh in ['pp_02', 'pp_05', 'pp_10', 'pp_15', 
               'pp_30', 'pp_45', 'pp_60']:

    dset_ = tor_area.sortby(tor_area[thresh], ascending=False)
    
    dtimes = [str(x.date()) for x in pd.to_datetime(dset_[thresh].time.values[:10])]
    areas = ["{:,}".format(int(1600*x)) for x in dset_[thresh].values[:10]]

    for date, area in list(zip(dtimes, areas)):
        
        if area == '0':
            dres[str(int(thresh[3:])) + "%"].append("-")
        else:
            dres[str(int(thresh[3:])) + "%"].append(date + " (" + area + ")")
        
df_table = pd.DataFrame.from_dict(dres)
df_table = df_table.set_index('Rank')
df_table[list(dres.keys())[1:]]

df_table.to_csv(out_dir + "sig_wind_top10_areas.csv")

df_table

,2%,5%,10%,15%,30%,45%,60%
Rank,,,,,,,
1,"2018-06-28 (1,756,800)","2018-06-28 (1,363,200)","2018-06-28 (1,016,000)","2012-06-29 (856,000)","2012-06-29 (643,200)","2012-06-29 (516,800)","2012-06-29 (448,000)"
2,"2016-06-17 (1,524,800)","2011-04-04 (1,185,600)","2012-06-29 (969,600)","2018-06-28 (836,800)","2018-06-28 (491,200)","1998-05-30 (387,200)","1998-05-30 (297,600)"
3,"2011-07-22 (1,521,600)","2012-06-29 (1,142,400)","2011-04-04 (876,800)","2011-04-04 (689,600)","1998-05-30 (491,200)","2018-06-28 (323,200)","2011-04-19 (249,600)"
4,"2011-04-04 (1,505,600)","2016-06-17 (1,108,800)","2011-04-19 (771,200)","2011-04-19 (656,000)","2011-04-19 (444,800)","2011-04-19 (320,000)","2014-06-30 (238,400)"
5,"2004-07-05 (1,465,600)","2004-07-05 (1,004,800)","1998-05-30 (732,800)","1998-05-30 (633,600)","2014-06-30 (409,600)","2014-06-30 (310,400)","2018-06-28 (235,200)"
6,"2011-07-10 (1,459,200)","1998-06-18 (1,001,600)","2014-06-30 (713,600)","2014-06-30 (600,000)","2011-04-04 (403,200)","1998-06-29 (268,800)","1994-07-01 (217,600)"
7,"2010-06-21 (1,433,600)","2012-07-01 (963,200)","2016-06-17 (676,800)","2017-03-06 (566,400)","2011-07-10 (368,000)","1994-07-01 (267,200)","1998-06-29 (208,000)"
8,"2008-06-08 (1,406,400)","2003-05-10 (960,000)","2017-03-06 (675,200)","2011-07-10 (553,600)","1998-06-29 (363,200)","1997-06-20 (262,400)","1997-06-20 (198,400)"
9,"1998-06-18 (1,396,800)","2008-06-08 (955,200)","2011-07-10 (649,600)","1998-06-29 (534,400)","2004-05-30 (348,800)","2011-07-10 (256,000)","2011-07-10 (192,000)"
